In [67]:
%%writefile ../services.py
#!/usr/local/bin/python 

#*** DO NOT EDIT - GENERATED FROM services.ipynb ****

import os
import pandas as pd
from  mangorest.mango import webapi

#*** new stuff for running inference ***
import cv2
import tensorflow as tf
import os # don't need this
import numpy as np
from django.http import HttpResponse
from PIL import Image
import io
import base64
from django.http import JsonResponse
import concurrent.futures
import math
import datetime
import paramiko


#Helper functions TODO:// Change username + password
def check_for_pothole(image_path="Images/test_image.png", model_path="Models/efficientLite0_Classifier.tflite", testing_mode=True):
    # Define class labels
    class_labels = {1: 'No Pothole', 2: 'Potholes Exist'}

    # Load the TFLite model using the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocess input image
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image.astype(input_details[0]['dtype'])

    # Set input tensor values
    interpreter.set_tensor(input_details[0]['index'], image)

    # Invoke interpreter
    interpreter.invoke()

    # Get output tensor values
    output = interpreter.get_tensor(output_details[0]['index'])

    # Postprocess output tensor
    prediction = np.argmax(output)
    class_label = class_labels[prediction + 1]

    # Display image if not on edge device
    # if(testing_mode == True):
    #     plt.imshow(image[0]/255)
    #     plt.title(class_label)
    #     plt.show()
    
    if(class_label == 'Potholes Exist'):
        return True
    else:
        return False
    
    import random



def get_random_location(latitude = 47.6062, longitude = -122.3321, radius_in_meters=1000):
    # Generate a random point within a square of side length 2r centered at (lat, long)
    lat_offset = radius_in_meters / 111300.0  # 1 degree of latitude is approximately 111300 meters
    long_offset = radius_in_meters / (111300.0 * abs(math.cos(latitude)))  # 1 degree of longitude is approximately 111300 meters at the equator
    lat = latitude + random.uniform(-lat_offset, lat_offset)
    long = longitude + random.uniform(-long_offset, long_offset)
    return (lat, long)

# Generate an ID to store the image based on datetime and location
def generate_id():
    random_location = get_random_location()
    current_datetime = datetime.datetime.now()
    current_datetime = current_datetime.strftime("%Y-%m-%d-%H-%M")
    return f"{random_location[0]}_{random_location[1]}_{current_datetime}"

#--SSH into linux server from edge devide 
# Ensure Global Connect, and wifi are connected on edge device
# If Pothole, send to remote queue
def submit_image_to_remote(image_path ="Images/test_image.png"): 
    '''
    Note: Our central server is hosted on a private server at SeattleU.
     If you wish to run this portion, you need to host your main server with 
     a hosting service such as AWS. You also need to adjust the submit_image_to_remote() 
     function and create an API to connect the two servers. After demonstrating our
     architecture, we have removed any personal information.
    '''
    
    # # Define a dictionary of file types
    # file_types = {
    #     '.pdf': '.pdf',
    #     '.jpg': '.jpg',
    #     '.png': '.png'
    # }

    # # Get the file path
    # file_path = image_path

    # # Get the file extension
    # file_extension = os.path.splitext(file_path)[1]

    # # Get the file type
    # if file_extension in file_types:
    #     file_type = file_types[file_extension]
    # else:
    #     file_type = 'Unknown File Type'
    
    
    # # Check if pothole exists
    # if(check_for_pothole(image_path=image_path)):
    #     ssh = paramiko.SSHClient()
    #     ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    #     ssh.connect('csp239.cslab..edu', username='', password='!')

    #     # Copy the image to the remote machine
    #     sftp = ssh.open_sftp()
        
    #     # Generate Filename 
    #     pothole_id = 'seattle:'+ generate_id() + file_type

    #     # Ensure Files and Remote Location Exist
    #     try:
    #         format_path = 'codebase/PotholeDetectionEdgeDevice/MainApp/PotholeDetector/PotholeQueue/{}'.format(pothole_id) 
    #         sftp.stat('codebase/PotholeDetectionEdgeDevice/MainApp/PotholeDetector/PotholeQueue/')
    #         sftp.put(image_path, format_path)
    #         print('Success')
    #     except IOError:
    #         print('Remote location doesnt exist' )

    #     # Close the SSH connection
    #     sftp.close()
    #     ssh.close()
    
    submit_image_to_remote(image_path="Images/RaspberryPi.png")
#--------------------------------------------------------------------------------------------------------
# Working Code *

# Inferene Handeler
def run_inference(image, model):
    image_exists = os.path.isfile(image)
    model_exists = os.path.isfile(model)
    print(f"\nrun_inference(image={image} ({image_exists}), model={model} ({model_exists}))\n")
   
    #TODO:// fix this for github
    if(check_for_pothole(image)):
        submit_image_to_remote(image)
        return True
    else:
        return False
    
    



#--------------------------------------------------------------------------------------------------------    
@webapi("/app1/test")
def test( request,  **kwargs):
    return "APP 1 TEST version 1.0"
#--------------------------------------------------------------------------------------------------------    
@webapi("/app1/uploadfile")
def uploadfile( request,  **kwargs):
    par = dict(request.GET)
    par.update(request.POST)
   # DESTDIR ="/tmp/MYAPP/" 
    DESTDIR = os.path.join("..", "dynamic")

    os.makedirs(DESTDIR, exist_ok=True)  # Create the destination directory if it does not exist

    ret = ""
    for file_ in request.FILES.getlist('file'):
        filename = str(file_)
        content = file_.read()
        file_path = os.path.join(DESTDIR, str(file_))
        with open(file_path, "wb") as f:
            f.write(content)
        ret += filename
        if os.path.isfile(file_path):  # Check if the file exists in the destination directory
            print(" saved successfully.")
        else:
            print("error uploading file")
    return ret
#--------------------------------------------------------------------------------------------------------    
@webapi("/app1/processfile")
def processfile(request, **kwargs):
    DESTDIR = os.path.join("..", "dynamic")
    filename = uploadfile(request, **kwargs)
    image_file = os.path.join(DESTDIR, filename)
    VGG = os.path.join('app1', 'Models/VGG.h5')
    output_path = run_inference(image_file, VGG)
    image = Image.open(output_path)
    # Apply a blue tint to the image
    blue_image = image.copy()
    # blue_image = blue_image.convert('RGB')
    # pixels = blue_image.load()
    # for i in range(blue_image.size[0]):
    #     for j in range(blue_image.size[1]):
    #         r, g, b = pixels[i, j]
    #         b = min(255, int(b * 1.5))
    #         pixels[i, j] = (r, g, b)
    # Save the modified image to a BytesIO object
    buffer = io.BytesIO()
    blue_image.save(buffer, format='JPEG')
    # Encode the image as a base64 string
    img_str = base64.b64encode(buffer.getvalue()).decode('utf-8')
    # Return the modified image as a JSON response
    return JsonResponse({'image_data': img_str})


Overwriting ../services.py
